# Fixing initial problems
The original regional averaging code didn't write the 0th element of every array and screwed up the time axis


In [1]:
import cdms2 as cdms
import MV2 as MV
import cdtime,cdutil,genutil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import string
import glob
import scipy.stats as stats
# Local solution
# If running remotely, uncomment the following code:
# %%bash
# git clone https://github.com/katemarvel/CMIP5_tools
# import CMIP5_tools as cmip5
import sys,os
sys.path.append("/Users/kmarvel/Google Drive/python-utils")

import CMIP5_tools as cmip5
import DA_tools
import Plotting

from eofs.cdms import Eof
from eofs.multivariate.cdms import MultivariateEof
%matplotlib inline

In [2]:
central_plains=cdutil.region.domain(longitude=(-105,-92),latitude=(32,46))
southwest=cdutil.region.domain(longitude=(-125,-105),latitude=(32,41))


In [3]:
#sshfs mout to kdm2144@dester.ldeo.columbia.edu:/home/kdm2144/ 
#datadirec="/Users/kmarvel/Documents/DATA/dester/"
datadirec="/Users/kmarvel/Documents/DATA/dester/piControlRaw/
#datadirec="/Volumes/RAVEN/CMIP6/cmip6rawyr/"
writedirec="/Users/kmarvel/Documents/DATA/dester/regional_averages/"
fixedvardirec="/Users/kmarvel/Documents/DATA/dester/fixedvar/"

In [4]:
#experiments=["historical","ssp126","ssp245","ssp370","ssp585"]
experiments="piControl"

In [6]:

variables=[x.split("/")[-1] for x in glob.glob(datadirec+"*")]
#variables=['mrsos', 'pr', 'prsn', 'tas']

###### LOOP OVER ALL VARIABLES #####
for variable in variables:
    
    modeldirs=glob.glob(datadirec+variable+"/*")
    ### LOOP OVER ALL MODELS
    for direc in modeldirs:
        model=direc.split("/")[-1]
        #print(model)
        allfiles=glob.glob(direc+"/*"+variable+"*")
        #experiments=np.unique([x.split(".")[1] for x in allfiles])
        #experiments=["historical","ssp126","ssp245","ssp370","ssp585"]
        experiments=["piControl"]
        ###### LOOP OVER ALL EXPERIMENTS #####
        for experiment in experiments:
            #print(experiment)
            cp_writedirec=writedirec+"central_plains/"+variable+"/"+experiment+"/"
            
            sw_writedirec=writedirec+"southwest/"+variable+"/"+experiment+"/"
            
            allfiles_experiment=glob.glob(direc+"/"+variable+"."+experiment+".*")
            rips=np.unique([x.split(".")[-3] for x in allfiles_experiment])

            landthresh=1
            #Get the land fraction
            landfiles=glob.glob(fixedvardirec+"sftlf*"+model+".*")
            

            if len(landfiles)==1:
                fland=cdms.open(landfiles[0])
                landfrac=fland("sftlf")
                fland.close()
            else:
                #print("can't find land fraction file for", model)
                #print(landfiles)
                continue


            ###### LOOP OVER ALL RIPS #####
            for rip in rips:
                #Open the existing data
                cp_writename=cp_writedirec+variable+"."+experiment+"."+model+"."+rip+".nc"
                fcpw=cdms.open(cp_writename)    
                CP=fcpw(variable)
                fcpw.close()
                sw_writename=sw_writedirec+variable+"."+experiment+"."+model+"."+rip+".nc"
                fsww=cdms.open(sw_writename)    
                SW = fsww(variable)
                fsww.close()  
                
                
                
                ripfiles=np.sort(glob.glob(direc+"/"+variable+"."+experiment+"."+model+"."+rip+"*"))
                print(variable+"."+experiment+"."+model+"."+rip+"*")
                L=len(ripfiles)
                i=0
                ripfile=ripfiles[i]
                frip=cdms.open(ripfile)
                data=frip(variable)
                frip.close()
                if data.shape[1:]!=landfrac.shape:
                    print ("land mask wrong shape for "+variable+"."+experiment+"."+model+"."+rip)
                    continue
                
                
                landdata=MV.masked_where(np.repeat(landfrac.asma()[np.newaxis],12,axis=0)<landthresh,data)
                fpath,fexpt,fmodel,frip,fyear,fnc=ripfile.split(".")
               
                CP[12*i:12*(i+1)]=cdutil.averager(landdata(central_plains),axis='xy')
                SW[12*i:12*(i+1)]=cdutil.averager(landdata(southwest),axis='xy')
             

                tax=cdms.createAxis(np.arange(L*12))
                tax.designateTime()
                tax.units='months since '+str(fyear)+'-1-1'
                CP.setAxis(0,tax)
                SW.setAxis(0,tax)
                CP.id=variable
                SW.id=variable
                cp_writename=cp_writedirec+variable+"."+experiment+"."+model+"."+rip+".nc"
                fcpw=cdms.open(cp_writename,"w")    
                fcpw.write(CP)
                fcpw.close()
                sw_writename=sw_writedirec+variable+"."+experiment+"."+model+"."+rip+".nc"
                fsww=cdms.open(sw_writename,"w")    
                fsww.write(SW)
                fsww.close()  

evspsbl.piControl.BCC-CSM2-MR.r1i1p1f1*
mrsos.piControl.BCC-ESM1.r1i1p1f1*
mrsos.piControl.CanESM5.r1i1p1f1*
mrsos.piControl.CanESM5.r1i1p2f1*
mrsos.piControl.CESM2.r1i1p1f1*
mrsos.piControl.CESM2-WACCM.r1i1p1f1*
mrsos.piControl.CNRM-ESM2-1.r1i1p1f2*


CDMSError: Cannot open file /Users/kmarvel/Documents/DATA/dester/regional_averages/central_plains/mrsos/piControl/mrsos.piControl.E3SM-1-0.r1i1p1f1.nc (Variable not found)

In [8]:
cp_writename

'/Users/kmarvel/Documents/DATA/dester/regional_averages/central_plains/mrsos/piControl/mrsos.piControl.E3SM-1-0.r1i1p1f1.nc'